Objetivo: Crear categorias para los columnas númericas de la tabla fact_weather para preparar-la para la modelización

In [0]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
import re

In [0]:
fact_weather_enriched = (spark.table("hive_metastore.tfm.fact_weather"))

In [0]:
display(spark.table("hive_metastore.tfm.fact_weather").limit(10))

weather_station_id,date,awnd,prcp,snow,snwd,tmax,tmin,wt01,wt02,wt03,wt04,wt05,wt06,wt08
USW00094728,2020-01-01,3.8,0.0,0.0,0.0,5.0,1.1,null,null,null,null,null,null,null
USW00094728,2020-01-02,2.4,0.0,0.0,0.0,9.4,0.6,null,null,null,null,null,null,null
USW00094728,2020-01-03,1.5,3.8,0.0,0.0,9.4,6.7,1,null,null,null,null,null,1
USW00094728,2020-01-04,2.0,6.9,0.0,0.0,10.6,5.0,1,1,null,null,null,null,null
USW00094728,2020-01-05,5.1,0.0,0.0,0.0,5.6,1.7,null,null,null,null,null,null,null
USW00094728,2020-01-06,2.7,1.0,5.0,0.0,7.2,0.0,1,null,null,null,null,null,null
USW00094728,2020-01-07,2.3,0.0,0.0,0.0,7.2,2.2,null,null,null,null,null,null,null
USW00094728,2020-01-08,4.3,0.0,0.0,0.0,5.6,-2.1,null,null,null,null,null,null,1
USW00094728,2020-01-09,2.0,0.0,0.0,0.0,1.1,-4.9,null,null,null,null,null,null,null
USW00094728,2020-01-10,2.8,0.0,0.0,0.0,12.8,0.6,null,null,null,null,null,null,null


1. Substituir nulls por 0

In [0]:
numeric_cols = [c for c, t in fact_weather_enriched.dtypes if t in ("int", "bigint", "float", "double")]

for c in numeric_cols:
    fact_weather_enriched = fact_weather_enriched.fillna({c: 0})


In [0]:
display(fact_weather_enriched.limit(10))

weather_station_id,date,awnd,prcp,snow,snwd,tmax,tmin,wt01,wt02,wt03,wt04,wt05,wt06,wt08
USW00094728,2020-01-01,3.8,0.0,0.0,0.0,5.0,1.1,0,0,0,0,0,0,0
USW00094728,2020-01-02,2.4,0.0,0.0,0.0,9.4,0.6,0,0,0,0,0,0,0
USW00094728,2020-01-03,1.5,3.8,0.0,0.0,9.4,6.7,1,0,0,0,0,0,1
USW00094728,2020-01-04,2.0,6.9,0.0,0.0,10.6,5.0,1,1,0,0,0,0,0
USW00094728,2020-01-05,5.1,0.0,0.0,0.0,5.6,1.7,0,0,0,0,0,0,0
USW00094728,2020-01-06,2.7,1.0,5.0,0.0,7.2,0.0,1,0,0,0,0,0,0
USW00094728,2020-01-07,2.3,0.0,0.0,0.0,7.2,2.2,0,0,0,0,0,0,0
USW00094728,2020-01-08,4.3,0.0,0.0,0.0,5.6,-2.1,0,0,0,0,0,0,1
USW00094728,2020-01-09,2.0,0.0,0.0,0.0,1.1,-4.9,0,0,0,0,0,0,0
USW00094728,2020-01-10,2.8,0.0,0.0,0.0,12.8,0.6,0,0,0,0,0,0,0


2.Crear nueva columna de temperatura

In [0]:
fact_weather_enriched = (
    fact_weather_enriched
    .withColumn('trange', F.col('tmax') - F.col('tmin'))
)

In [0]:
display(fact_weather_enriched.limit(10))

weather_station_id,date,awnd,prcp,snow,snwd,tmax,tmin,wt01,wt02,wt03,wt04,wt05,wt06,wt08,trange
USW00094789,2020-01-01,28.437108480000006,0.0,0.0,0.0,5.555555555555555,1.1111111111111112,0,0,0,0,0,0,0,4.444444444444445
USW00094789,2020-01-02,15.481889279999999,0.0,0.0,0.0,8.88888888888889,-0.5555555555555556,0,0,0,0,0,0,0,9.444444444444445
USW00094789,2020-01-03,10.444642560000002,3.302,0.0,0.0,8.88888888888889,5.555555555555555,1,0,0,0,0,0,0,3.333333333333334
USW00094789,2020-01-04,13.325368319999999,4.064,0.0,0.0,10.555555555555555,6.111111111111112,1,1,0,0,0,0,0,4.444444444444444
USW00094789,2020-01-05,34.55261568,0.0,0.0,0.0,9.444444444444445,1.6666666666666667,0,0,0,0,0,0,0,7.777777777777778
USW00094789,2020-01-06,22.32160128,0.7619999999999999,0.3,0.0,7.222222222222222,0.0,1,0,0,0,0,0,0,7.222222222222222
USW00094789,2020-01-07,13.679424000000001,0.7619999999999999,0.0,0.0,6.666666666666667,1.6666666666666667,0,0,0,0,0,0,0,5.0
USW00094789,2020-01-08,33.474355200000005,0.0,0.0,0.0,6.111111111111112,-2.2222222222222223,0,0,0,0,0,0,0,8.333333333333334
USW00094789,2020-01-09,17.99246592,0.0,0.0,0.0,1.1111111111111112,-4.444444444444445,0,0,0,0,0,0,0,5.555555555555555
USW00094789,2020-01-10,12.601163520000002,0.0,0.0,0.0,10.555555555555555,0.0,0,0,0,0,0,0,0,10.555555555555555


2. Calcular splits

In [0]:
#COLUMNA AWND

# Vector de características
assembler = VectorAssembler(
    inputCols=['awnd'],
    outputCol='features_awnd',
    handleInvalid="keep"
)

# Árbol de decisión
dt = DecisionTreeRegressor(
    labelCol='awnd',
    featuresCol='features_awnd',
    maxDepth=3
)

# Entrenar modelo
pipeline = Pipeline(stages=[assembler, dt])

model = pipeline.fit(fact_weather_enriched)

tree_model = model.stages[-1]

# Splits
tree_text = tree_model.toDebugString

splits = sorted({float(x) for x in re.findall(r'<= ([0-9\.]+)', tree_text)})

print('Splits:', splits)

Splits: [2.05, 4.69159552, 9.18130752, 12.062033280000001, 16.021019520000003, 21.058266240000002, 26.103559679999996]


In [0]:
#COLUMNA PRCP

# Vector de características
assembler = VectorAssembler(
    inputCols=['prcp'],
    outputCol='features_prcp',
    handleInvalid="keep"
)

# Árbol de decisión
dt = DecisionTreeRegressor(
    labelCol='prcp',
    featuresCol='features_prcp',
    maxDepth=3
)

# Entrenar modelo
pipeline = Pipeline(stages=[assembler, dt])

model = pipeline.fit(fact_weather_enriched)

tree_model = model.stages[-1]

# Splits
tree_text = tree_model.toDebugString

splits = sorted({float(x) for x in re.findall(r'<= ([0-9\.]+)', tree_text)})

print('Splits:', splits)

Splits: [1.9, 5.821, 11.415, 16.637, 25.122999999999998]


In [0]:
#COLUMNA SNOW

# Vector de características
assembler = VectorAssembler(
    inputCols=['snow'],
    outputCol='features_snow',
    handleInvalid="keep"
)

# Árbol de decisión
dt = DecisionTreeRegressor(
    labelCol='snow',
    featuresCol='features_snow',
    maxDepth=3
)

# Entrenar modelo
pipeline = Pipeline(stages=[assembler, dt])

model = pipeline.fit(fact_weather_enriched)

tree_model = model.stages[-1]

# Splits
tree_text = tree_model.toDebugString

splits = sorted({float(x) for x in re.findall(r'<= ([0-9\.]+)', tree_text)})

print('Splits:', splits)

Splits: [0.25, 1.05, 2.3499999999999996, 4.1]


In [0]:
#COLUMNA SNWD

# Vector de características
assembler = VectorAssembler(
    inputCols=['snwd'],
    outputCol='features_snwd',
    handleInvalid="keep"
)

# Árbol de decisión
dt = DecisionTreeRegressor(
    labelCol='snwd',
    featuresCol='features_snwd',
    maxDepth=3
)

# Entrenar modelo
pipeline = Pipeline(stages=[assembler, dt])

model = pipeline.fit(fact_weather_enriched)

tree_model = model.stages[-1]

# Splits
tree_text = tree_model.toDebugString

splits = sorted({float(x) for x in re.findall(r'<= ([0-9\.]+)', tree_text)})

print('Splits:', splits)

Splits: [3.5, 23.45, 65.0, 90.0, 165.0, 215.0, 305.0]


In [0]:
#COLUMNA TMAX

# Vector de características
assembler = VectorAssembler(
    inputCols=['tmax'],
    outputCol='features_tmax',
    handleInvalid="keep"
)

# Árbol de decisión
dt = DecisionTreeRegressor(
    labelCol='tmax',
    featuresCol='features_tmax',
    maxDepth=3
)

# Entrenar modelo
pipeline = Pipeline(stages=[assembler, dt])

model = pipeline.fit(fact_weather_enriched)

tree_model = model.stages[-1]

# Splits
tree_text = tree_model.toDebugString

splits = sorted({float(x) for x in re.findall(r'<= ([0-9\.]+)', tree_text)})

print('Splits:', splits)

Splits: [3.6111111111111116, 9.15, 13.316666666666666, 17.5, 21.683333333333334, 25.57777777777778, 30.85]


In [0]:
#COLUMNA TMIN

# Vector de características
assembler = VectorAssembler(
    inputCols=['tmin'],
    outputCol='features_tmin',
    handleInvalid="keep"
)

# Árbol de decisión
dt = DecisionTreeRegressor(
    labelCol='tmin',
    featuresCol='features_tmin',
    maxDepth=3
)

# Entrenar modelo
pipeline = Pipeline(stages=[assembler, dt])

model = pipeline.fit(fact_weather_enriched)

tree_model = model.stages[-1]

# Splits
tree_text = tree_model.toDebugString

splits = sorted({float(x) for x in re.findall(r'<= ([0-9\.]+)', tree_text)})

print('Splits:', splits)

Splits: [2.2111111111111112, 6.105555555555556, 10.277777777777779, 13.894444444444446, 17.78888888888889, 21.38888888888889]


In [0]:
#COLUMNA TRANGE

# Vector de características
assembler = VectorAssembler(
    inputCols=['trange'],
    outputCol='features_trange',
    handleInvalid="keep"
)

# Árbol de decisión
dt = DecisionTreeRegressor(
    labelCol='trange',
    featuresCol='features_trange',
    maxDepth=3
)

# Entrenar modelo
pipeline = Pipeline(stages=[assembler, dt])

model = pipeline.fit(fact_weather_enriched)

tree_model = model.stages[-1]

# Splits
tree_text = tree_model.toDebugString

splits = sorted({float(x) for x in re.findall(r'<= ([0-9\.]+)', tree_text)})

print('Splits:', splits)

Splits: [3.9000000000000012, 5.5777777777777775, 7.222222222222223, 8.366666666666667, 10.0, 11.66666666666667, 14.444444444444446]


3. Categorizar columnas

In [0]:
fact_weather_enriched = (
    fact_weather_enriched
    .withColumn(
        'awnd_bin',
        F.when((F.col('awnd') <= 2.05), 'cat0')
        .when((F.col('awnd') <= 4.69159552), 'cat1')
        .when((F.col('awnd') <= 9.18130752), 'cat2')
        .when((F.col('awnd') <= 12.062033280000001), 'cat3')
        .when((F.col('awnd') <= 16.021019520000003), 'cat4')
        .when((F.col('awnd') <= 21.058266240000002), 'cat5')
        .when((F.col('awnd') <= 26.103559679999996), 'cat6')
        .otherwise('cat7')
    )
    .withColumn(
        'prcp_bin',
        F.when((F.col('prcp') <= 1.9), 'cat0')
        .when((F.col('prcp') <= 5.821), 'cat1')
        .when((F.col('prcp') <= 11.415), 'cat2')
        .when((F.col('prcp') <= 16.637), 'cat3')
        .when((F.col('prcp') <= 25.122999999999998), 'cat4')
        .otherwise('cat5')
    )
    .withColumn(
        'snow_bin',
        F.when((F.col('snow') <= 0.25), 'cat0')
        .when((F.col('snow') <= 1.05), 'cat1')
        .when((F.col('snow') <= 2.3499999999999996), 'cat2')
        .when((F.col('snow') <= 4.1), 'cat3')
        .otherwise('cat4')
    )
    .withColumn(
        'snwd_bin',
        F.when((F.col('snwd') <= 3.5), 'cat0')
        .when((F.col('snwd') <= 23.45), 'cat1')
        .when((F.col('snwd') <= 65), 'cat2')
        .when((F.col('snwd') <= 90), 'cat3')
        .when((F.col('snwd') <= 165), 'cat4')
        .when((F.col('snwd') <= 215), 'cat5')
        .when((F.col('snwd') <= 305), 'cat6')
        .otherwise('cat7')
    )
    .withColumn(
        'tmax_bin',
        F.when((F.col('tmax') <= 3.6111111111111116), 'cat0')
        .when((F.col('tmax') <= 9.15), 'cat1')
        .when((F.col('tmax') <= 13.316666666666666), 'cat2')
        .when((F.col('tmax') <= 17.5), 'cat3')
        .when((F.col('tmax') <= 21.683333333333334), 'cat4')
        .when((F.col('tmax') <= 25.57777777777778), 'cat5')
        .when((F.col('tmax') <= 30.85), 'cat6')
        .otherwise('cat7')
    )
    .withColumn(
        'tmin_bin',
        F.when((F.col('tmin') <= 2.2111111111111112), 'cat0')
        .when((F.col('tmin') <= 6.105555555555556), 'cat1')
        .when((F.col('tmin') <= 10.277777777777779), 'cat2')
        .when((F.col('tmin') <= 13.894444444444446), 'cat3')
        .when((F.col('tmin') <= 17.78888888888889), 'cat4')
        .when((F.col('tmin') <= 21.38888888888889), 'cat5')
        .otherwise('cat6')
    )
    .withColumn(
        'trange_bin',
        F.when((F.col('trange') <= 3.9000000000000012), 'cat0')
        .when((F.col('trange') <= 5.5777777777777775), 'cat1')
        .when((F.col('trange') <= 7.222222222222223), 'cat2')
        .when((F.col('trange') <= 8.366666666666667), 'cat3')
        .when((F.col('trange') <= 10.0), 'cat4')
        .when((F.col('trange') <= 11.66666666666667), 'cat5')
        .when((F.col('trange') <= 14.444444444444446), 'cat6')
        .otherwise('cat7')
    )    
)

In [0]:
display(fact_weather_enriched.limit(10))

weather_station_id,date,awnd,prcp,snow,snwd,tmax,tmin,wt01,wt02,wt03,wt04,wt05,wt06,wt08,trange,awnd_bin,prcp_bin,snow_bin,snwd_bin,tmax_bin,tmin_bin,trange_bin
USW00094789,2020-01-01,28.437108480000006,0.0,0.0,0.0,5.555555555555555,1.1111111111111112,0,0,0,0,0,0,0,4.444444444444445,cat7,cat0,cat0,cat0,cat1,cat0,cat1
USW00094789,2020-01-02,15.481889279999999,0.0,0.0,0.0,8.88888888888889,-0.5555555555555556,0,0,0,0,0,0,0,9.444444444444445,cat4,cat0,cat0,cat0,cat1,cat0,cat4
USW00094789,2020-01-03,10.444642560000002,3.302,0.0,0.0,8.88888888888889,5.555555555555555,1,0,0,0,0,0,0,3.333333333333334,cat3,cat1,cat0,cat0,cat1,cat1,cat0
USW00094789,2020-01-04,13.325368319999999,4.064,0.0,0.0,10.555555555555555,6.111111111111112,1,1,0,0,0,0,0,4.444444444444444,cat4,cat1,cat0,cat0,cat2,cat2,cat1
USW00094789,2020-01-05,34.55261568,0.0,0.0,0.0,9.444444444444445,1.6666666666666667,0,0,0,0,0,0,0,7.777777777777778,cat7,cat0,cat0,cat0,cat2,cat0,cat3
USW00094789,2020-01-06,22.32160128,0.7619999999999999,0.3,0.0,7.222222222222222,0.0,1,0,0,0,0,0,0,7.222222222222222,cat6,cat0,cat1,cat0,cat1,cat0,cat2
USW00094789,2020-01-07,13.679424000000001,0.7619999999999999,0.0,0.0,6.666666666666667,1.6666666666666667,0,0,0,0,0,0,0,5.0,cat4,cat0,cat0,cat0,cat1,cat0,cat1
USW00094789,2020-01-08,33.474355200000005,0.0,0.0,0.0,6.111111111111112,-2.2222222222222223,0,0,0,0,0,0,0,8.333333333333334,cat7,cat0,cat0,cat0,cat1,cat0,cat3
USW00094789,2020-01-09,17.99246592,0.0,0.0,0.0,1.1111111111111112,-4.444444444444445,0,0,0,0,0,0,0,5.555555555555555,cat5,cat0,cat0,cat0,cat0,cat0,cat1
USW00094789,2020-01-10,12.601163520000002,0.0,0.0,0.0,10.555555555555555,0.0,0,0,0,0,0,0,0,10.555555555555555,cat4,cat0,cat0,cat0,cat2,cat0,cat5


4. Crear columna concatenada para el JOIN con las tablas de transporte

In [0]:
fact_weather_enriched = (
    fact_weather_enriched
        .withColumn(
            'weather_station_date',
            F.concat_ws('/', F.col('weather_station_id'), F.col('date'))
        )
)

In [0]:
display(fact_weather_enriched.limit(10))

weather_station_id,date,awnd,prcp,snow,snwd,tmax,tmin,wt01,wt02,wt03,wt04,wt05,wt06,wt08,trange,awnd_bin,prcp_bin,snow_bin,snwd_bin,tmax_bin,tmin_bin,trange_bin,weather_station_date
USW00094789,2020-01-01,28.437108480000006,0.0,0.0,0.0,5.555555555555555,1.1111111111111112,0,0,0,0,0,0,0,4.444444444444445,cat7,cat0,cat0,cat0,cat1,cat0,cat1,USW00094789/2020-01-01
USW00094789,2020-01-02,15.481889279999999,0.0,0.0,0.0,8.88888888888889,-0.5555555555555556,0,0,0,0,0,0,0,9.444444444444445,cat4,cat0,cat0,cat0,cat1,cat0,cat4,USW00094789/2020-01-02
USW00094789,2020-01-03,10.444642560000002,3.302,0.0,0.0,8.88888888888889,5.555555555555555,1,0,0,0,0,0,0,3.333333333333334,cat3,cat1,cat0,cat0,cat1,cat1,cat0,USW00094789/2020-01-03
USW00094789,2020-01-04,13.325368319999999,4.064,0.0,0.0,10.555555555555555,6.111111111111112,1,1,0,0,0,0,0,4.444444444444444,cat4,cat1,cat0,cat0,cat2,cat2,cat1,USW00094789/2020-01-04
USW00094789,2020-01-05,34.55261568,0.0,0.0,0.0,9.444444444444445,1.6666666666666667,0,0,0,0,0,0,0,7.777777777777778,cat7,cat0,cat0,cat0,cat2,cat0,cat3,USW00094789/2020-01-05
USW00094789,2020-01-06,22.32160128,0.7619999999999999,0.3,0.0,7.222222222222222,0.0,1,0,0,0,0,0,0,7.222222222222222,cat6,cat0,cat1,cat0,cat1,cat0,cat2,USW00094789/2020-01-06
USW00094789,2020-01-07,13.679424000000001,0.7619999999999999,0.0,0.0,6.666666666666667,1.6666666666666667,0,0,0,0,0,0,0,5.0,cat4,cat0,cat0,cat0,cat1,cat0,cat1,USW00094789/2020-01-07
USW00094789,2020-01-08,33.474355200000005,0.0,0.0,0.0,6.111111111111112,-2.2222222222222223,0,0,0,0,0,0,0,8.333333333333334,cat7,cat0,cat0,cat0,cat1,cat0,cat3,USW00094789/2020-01-08
USW00094789,2020-01-09,17.99246592,0.0,0.0,0.0,1.1111111111111112,-4.444444444444445,0,0,0,0,0,0,0,5.555555555555555,cat5,cat0,cat0,cat0,cat0,cat0,cat1,USW00094789/2020-01-09
USW00094789,2020-01-10,12.601163520000002,0.0,0.0,0.0,10.555555555555555,0.0,0,0,0,0,0,0,0,10.555555555555555,cat4,cat0,cat0,cat0,cat2,cat0,cat5,USW00094789/2020-01-10


5. Guardar tabla

In [0]:
fact_weather_enriched.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("tfm.fact_weather_enriched")